In [5]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import warnings
import string
warnings.filterwarnings('ignore')

# NHL
## Read in data

In [2]:
nhl_draft=pd.read_csv('..\\..\\Data\\stata-files\\nhl_draft.csv')

## Merge individual season csv's

In [ ]:
skaters=pd.read_csv('..\\Data\\NHL-Stats-0922\\skaters\\skaters_2008.csv')
goalies=pd.read_csv('..\\Data\\NHL-Stats-0922\\goalies\\goalies_2008.csv')

for i in range(2009,2023):
    sk=pd.read_csv('..\\Data\\NHL-Stats-0922\\skaters\\skaters_'+str(i)+'.csv')
    gl=pd.read_csv('..\\Data\\NHL-Stats-0922\\goalies\\goalies_'+str(i)+'.csv')
    
    skaters=skaters.append(sk)
    goalies=goalies.append(gl)

skaters.to_csv('..\\Data\\NHL-Stats-0922\\skaters.csv',index=False)
goalies.to_csv('..\\Data\\NHL-Stats-0922\\goalies.csv',index=False)

## Create NHL round 

In [35]:
teams={1979:21,1991:22,1992:24,1993:26,1998:27,1999:28,2000:30,2017:31,2021:32}
num_rounds={}
for i in range(1982,2022):
    if i<1992:
        num_rounds[i]=12
    elif i < 1995:
        num_rounds[i]=11
    elif i < 2005:
        num_rounds[i]=9
    else:
        num_rounds[i]=7
nhl_draft=nhl_draft[nhl_draft['year']>=1982]

In [45]:
for index, row in nhl_draft.iterrows():
    if  row['overall_pick']+1!=nhl_draft.iloc[index+1]['overall_pick'] and row['year']==nhl_draft.iloc[index+1]['year']:
        forfeit=pd.DataFrame(({'id':0, 'overall_pick':row['overall_pick']+1,'team':'Forfeit', 'year':row['year']}), index=[0])
        nhl_draft=pd.concat([nhl_draft.iloc[:index+1],forfeit,nhl_draft.iloc[index+1:]], ignore_index=True)

In [46]:
round=[]
for index, row in nhl_draft.iterrows():
    num_teams=len(nhl_draft[nhl_draft['year']==row['year']].team.unique())
    overall=row['overall_pick']-1
    r=overall//(num_teams)+1
    round.append(r)
    comp_picks=nhl_draft[nhl_draft['year']==row['year']].overall_pick.max()%num_teams
    if comp_picks!=0:
        if round[-1]==3 and comp_picks>round.count(2)-num_teams:
            round[-1]=2
        elif round[-1]>3 and round.count(r-1)<num_teams:
            round[-1]=r-1
nhl_draft['round']=round

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(nhl_draft[['id','year','overall_pick','round']])

## Create pick, and running variable (distRound)

In [47]:
rounds=nhl_draft['round'].to_numpy()
prev=0
pick=1
picks=[]
for r in rounds:
    if r==prev:
        pick+=1
    else:
        pick=1

    prev=r
    picks.append(pick)


nhl_draft['pick']=picks

distRound=[]
for index, row in nhl_draft.iterrows():
    numPicks=nhl_draft[(nhl_draft['year']==row['year']) & (nhl_draft['round']==row['round'])]['pick'].max()
    m=numPicks//2
    if row['pick']>m:
        distRound.append(row['pick']-numPicks)
    else:
        distRound.append(row['pick'])

nhl_draft['distRound']=distRound

## Fill NHL NA's

In [49]:
nhl_draft[['to_year']]=nhl_draft[['to_year']].fillna(2022)
nhl_draft[['games_played']]=nhl_draft[['games_played']].fillna(0)
nhl_draft=nhl_draft.fillna(0)

In [52]:
nhl_draft=nhl_draft[nhl_draft['id']!=0]

## Create years played and age of retirement

In [50]:
nhl_draft['Years_Played']= nhl_draft['to_year']-nhl_draft['year']
nhl_draft['age_of_retirement']=nhl_draft['age']+nhl_draft['Years_Played']

## Condense position codes

In [7]:
for index, row in nhl_draft.iterrows():
    if type(row['position'])==float:
        nhl_draft.drop(index, inplace=True)

positions = nhl_draft['position'].unique()
positions

array(['LW', 'D', 'C', 'RW', 'G', 'W', 'L', 'F'], dtype=object)

In [8]:
positions=nhl_draft['position'].to_numpy()
for i in range(len(positions)):
        
    if len(positions[i])==1:
        pass
    else:
        positions[i]=positions[i][:2].strip('/;')
    if positions[i]=='Ce':
        positions[i]='C'
nhl_draft['position']=positions
nhl_draft['position'].unique()

array(['LW', 'D', 'C', 'RW', 'G', 'W', 'L', 'F'], dtype=object)

In [9]:
## Ordinal Encode Position
ord_enc = OrdinalEncoder()
nhl_draft["position_code"] = ord_enc.fit_transform(nhl_draft[["position"]])

## Standardize the data

In [53]:
## Scale by position
scaler=StandardScaler()
pos_scaled_nhl_draft=pd.DataFrame()
for pos in nhl_draft['position'].unique():
    temp=nhl_draft[nhl_draft['position']==pos].copy()
    temp['Years_Played']=scaler.fit_transform(temp[['Years_Played']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games_played']=scaler.fit_transform(temp[['games_played']])
    pos_scaled_nhl_draft=pd.concat([pos_scaled_nhl_draft, temp])


In [54]:
## Scale by round
scaler=StandardScaler()
round_scaled_nhl_draft=pd.DataFrame()
for round in nhl_draft['round'].unique():
    temp=nhl_draft[nhl_draft['round']==round].copy()
    temp['Years_Played']=scaler.fit_transform(temp[['Years_Played']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games_played']=scaler.fit_transform(temp[['games_played']])    
    round_scaled_nhl_draft=pd.concat([round_scaled_nhl_draft, temp])

In [55]:
## Scale by round and position
scaler=StandardScaler()
scaled_nhl_draft=pd.DataFrame()
for round in pos_scaled_nhl_draft['round'].unique():
    temp=pos_scaled_nhl_draft[pos_scaled_nhl_draft['round']==round].copy()
    temp['Years_Played']=scaler.fit_transform(temp[['Years_Played']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games_played']=scaler.fit_transform(temp[['games_played']])
    scaled_nhl_draft=pd.concat([scaled_nhl_draft, temp])

In [56]:
nhl_draft['pos_years']=pos_scaled_nhl_draft['Years_Played']
nhl_draft['pos_age_of_retirement']=pos_scaled_nhl_draft['age_of_retirement']
nhl_draft['pos_games_played']=pos_scaled_nhl_draft['games_played']


In [57]:
nhl_draft['round_years']=round_scaled_nhl_draft['Years_Played']
nhl_draft['round_age_of_retirement']=round_scaled_nhl_draft['age_of_retirement']
nhl_draft['round_games_played']=round_scaled_nhl_draft['games_played']

In [58]:
nhl_draft['scaled_years']=scaled_nhl_draft['Years_Played']
nhl_draft['scaled_age_of_retirement']=scaled_nhl_draft['age_of_retirement']
nhl_draft['scaled_games_played']=scaled_nhl_draft['games_played']

## Write to CSV

In [41]:
nhl_draft.to_csv('..\\..\\Data\\stata-files\\nhl_draft.csv',index=False)


## Add season level data

### Read in data

In [30]:
skaters = pd.read_csv('..\\..\\Data\\NHL-Stats-0922\\skaters.csv')
goalies = pd.read_csv('..\\..\\Data\\NHL-Stats-0922\\goalies.csv')
nhl_draft=pd.read_csv('..\\..\\Data\\stata-files\\nhl_draft.csv')

In [31]:
skaters = skaters[skaters['situation']=='all']
goalies = goalies[goalies['situation']=='all']
#nhl_draft['playerID'] = np.empty(len(nhl_draft))
skaters_draft = nhl_draft[nhl_draft['position']!='G']
goalies_draft = nhl_draft[nhl_draft['position']=='G']
goalies.to_csv('..\\..\\Data\\stata-files\\goalies_cond.csv',index=False)
skaters.to_csv('..\\..\\Data\\stata-files\\skaters_cond.csv',index=False)

In [ ]:
undrafted = []
nhl_draft_names = nhl_draft['player'].unique()
for index, row in skaters.iterrows():
    if row['name'] not in nhl_draft_names:
        undrafted.append(row['name'])

undrafted

### Add player ID to draft dataset to facilitate join

In [31]:
skater_names = skaters['name'].value_counts()
duplicate_names = []
skater_names = skater_names.to_dict()
for name in skater_names.keys():
    df = skaters[skaters['name']==name]
    if len(df['playerId'].unique())>1:
        duplicate_names.append(name)

duplicate_names

['Colin White',
 'Sebastian Aho',
 'Sean Collins',
 'Alexandre Picard',
 'Nathan Smith',
 'Mikko Lehtonen']

In [30]:
for index, row in skaters.iterrows():
    if row['name'] == 'Erik Gustafsson':
        skaters.at[index, 'playerId'] = 8475640

In [27]:
skaters.iloc[971]

playerId                              8471887
season                                   2009
name                         Patric Hornqvist
team                                      NSH
position                                    R
                                   ...       
xGoalsAgainstAfterShifts                  0.0
corsiForAfterShifts                       0.0
corsiAgainstAfterShifts                   0.0
fenwickForAfterShifts                     0.0
fenwickAgainstAfterShifts                 0.0
Name: 971, Length: 154, dtype: object

In [32]:
for index, row in skaters.iterrows():
    #name, postion, team
    if row['name']=='Nathan Smith':
        if row['team']=='MIN':
            nhl_draft.at[index, 'playerID'] =  row['playerId']
        else:
            nhl_draft.at[index, 'playerID'] = row['playerId']



    else:
        if row['name'] not in duplicate_names:
            temp = nhl_draft[nhl_draft['player']==row['name']]
            if len(temp)==1:
                nhl_draft.at[temp.index[0], 'playerID'] = row['playerId']
            elif len(temp)>1:
                for id in temp.index:
                    nhl_draft.at[id, 'playerID'] = row['playerId']
        else:
            temp = nhl_draft[(nhl_draft['player']==row['name']) & (nhl_draft['position']==row['position'])]
            if len(temp)==1:
                nhl_draft.at[temp.index[0], 'playerID'] = row['playerId']
            elif len(temp)>1:
                for id in temp.index:
                    nhl_draft.at[id, 'playerID'] = row['playerId']


In [8]:
duplicate_names = []

goalie_names = goalies['name'].value_counts()
goalie_names = goalie_names.to_dict()

for name in goalie_names.keys():
    df = goalies[goalies['name']==name]
    if len(df['playerId'].unique())>1:
        duplicate_names.append(name)

duplicate_names

['Matt Murray']

In [34]:
goalies = pd.read_csv('..\\..\\Data\\goalies_cond.csv')

In [35]:
for index, row in goalies.iterrows():
    if row['name'] == 'Matt Murray':
        nhl_draft.at[2240, 'playerID'] =  8476899
    else:
        temp = nhl_draft[nhl_draft['player']==row['name']]
        if len(temp)>0:
            nhl_draft.at[temp.index[0], 'playerID'] = row['playerId']
    

In [36]:
nhl_draft.position = nhl_draft.position.replace({'LW':'L', 'RW':'R', 'C':'C', 'D':'D', 'G':'G'})

In [37]:
#nhl_draft.at[724,'playerID']=8481537
nhl_draft[(nhl_draft['playerID']==0) & (nhl_draft['to_year']>2008) & (nhl_draft['games_played']>0) & (nhl_draft['position']!='G')][['year','player','position','team','to_year','games_played', 'playerID']]

,year,player,position,team,to_year,games_played,playerID
1865,2014.0,Clinston Franklin,L,Winnipeg Jets,2022.0,4.0,0.0


In [38]:
skaters = pd.read_csv('..\\..\\Data\\skaters_cond.csv')
skaters['name']=skaters['name'].str.upper()
for index, row in nhl_draft.iterrows():
    if (row['to_year']>2008) and (row['games_played']>0) and (row['playerID']==0):
        last_name=row['player'].split(' ')[-1]
        last_name=last_name.upper()
        temp=skaters[skaters['name'].str.contains(last_name)]
        if len(temp.playerId.unique())==1:
            nhl_draft.at[index, 'playerID']=temp['playerId'].values[0]

In [39]:
for index, row in nhl_draft.iterrows():
    if (row['to_year']>2008) and (row['games_played']>0) and (row['playerID']==0):
        name = row['player'].replace('.', '')
        name = name.upper()
        temp=skaters[skaters['name'].str.contains(name)]
        if len(temp.playerId.unique())==1:
            nhl_draft.at[index, 'playerID']=temp['playerId'].values[0]

In [40]:
nhl_draft.playerID=nhl_draft.playerID.astype(int)

### Add draft and career data to season data

In [3]:
## games played, years played, age of retirement, scaled versions, round, pick, overall pick, position code, distRound
columns = ['year','round','pick','overall_pick','position_code','distRound']
combined_skaters = pd.read_csv('..\\..\\Data\\skaters_cond.csv')
combined_goalies = pd.read_csv('..\\..\\Data\\goalies_cond.csv')

for col in columns:
    combined_skaters[col]=np.nan
    combined_goalies[col]=np.nan

In [4]:
for index, row in combined_skaters.iterrows():
    temp = nhl_draft[nhl_draft['playerID']==row['playerId']]
    for col in columns:
        if len(temp)>0:
            combined_skaters.at[index,col]=temp[col].values[0]


In [5]:
for index, row in combined_goalies.iterrows():
    temp = nhl_draft[nhl_draft['playerID']==row['playerId']]
    for col in columns:
        if len(temp)>0:
            combined_goalies.at[index,col]=temp[col].values[0]

In [11]:
combined_skaters['age']=np.nan
combined_goalies['age']=np.nan

for index, row in combined_skaters.iterrows():
    temp = nhl_draft[nhl_draft['playerID']==row['playerId']]
    if len(temp)>0:
        combined_skaters.at[index,'age']=temp['age'].values[0]+(row['season']-row['year'])

for index, row in combined_goalies.iterrows():
    temp = nhl_draft[nhl_draft['playerID']==row['playerId']]
    if len(temp)>0:
        combined_goalies.at[index,'age']=temp['age'].values[0]+(row['season']-row['year'])

In [28]:
combined_skaters.to_csv('..\\..\\Data\\stata-files\\skaters_cmb.csv',index=False)
combined_goalies.to_csv('..\\..\\Data\\stata-files\\goalies_cmb.csv',index=False)

### Generate stats after age 24

In [13]:
combined_skaters = pd.read_csv('..\\..\\Data\\stata-files\\skaters_cmb.csv')
combined_goalies = pd.read_csv('..\\..\\Data\\stata-files\\goalies_cmb.csv')

In [14]:
combined_skaters['games_over_24']=np.nan
combined_skaters['seasons_over_24']=np.nan
combined_goalies['games_over_24']=np.nan
combined_goalies['seasons_over_24']=np.nan
combined_skaters['age_at_retirement']=np.nan
combined_goalies['age_at_retirement']=np.nan

In [27]:
games_over_24 = {}
seasons_over_24 = {}
age_at_retirement = {}

for player in combined_skaters['playerId'].unique():
    temp = combined_skaters[combined_skaters['playerId']==player]
    if temp.empty==False:
        games_over_24[player]=temp[temp['age']>24]['games_played'].sum()
        seasons_over_24[player]=len(temp[temp['age']>24])
        retirement=temp['age'].max()
        if temp[temp['age']==retirement]['season'].values<2022:
            age_at_retirement[player]=retirement
        else:
            age_at_retirement[player]=np.nan

for player in combined_goalies['playerId'].unique():
    temp = combined_goalies[combined_goalies['playerId']==player]
    if temp.empty==False:
        games_over_24[player]=temp[temp['age']>24]['games_played'].sum()
        seasons_over_24[player]=len(temp[temp['age']>24])
        retirement=temp['age'].max()
        if temp[temp['age']==retirement]['season'].values<2022:
            age_at_retirement[player]=retirement
        else:
            age_at_retirement[player]=np.nan


for index, row in combined_skaters.iterrows():
    if row['playerId'] in games_over_24.keys():
        combined_skaters.at[index,'games_over_24']=games_over_24[row['playerId']]
        combined_skaters.at[index,'seasons_over_24']=seasons_over_24[row['playerId']]
        combined_skaters.at[index,'age_at_retirement']=age_at_retirement[row['playerId']]


for index, row in combined_goalies.iterrows():
    if row['playerId'] in games_over_24.keys():
        combined_goalies.at[index,'games_over_24']=games_over_24[row['playerId']]
        combined_goalies.at[index,'seasons_over_24']=seasons_over_24[row['playerId']]
        combined_goalies.at[index,'age_at_retirement']=age_at_retirement[row['playerId']]


### Drop unneeded variables and merge goalies and skaters

In [29]:
combined_skaters = combined_skaters[['playerId','name','team','position','season','round','pick','overall_pick','age','year',
                                     'position_code','distRound','games_played','games_over_24','seasons_over_24','age_at_retirement']]
combined_goalies = combined_goalies[['playerId','name','team','position','season','round','pick','overall_pick','age','year',
                                     'position_code','distRound','games_played','games_over_24','seasons_over_24','age_at_retirement']]
combined_skaters.rename(columns={'year':'draft_year'}, inplace=True)
combined_goalies.rename(columns={'year':'draft_year'}, inplace=True)

combined = pd.concat([combined_skaters, combined_goalies], ignore_index=True)
combined.sort_values(by=['season', 'name'], inplace=True)
combined.to_csv('..\\..\\Data\\stata-files\\combined.csv',index=False)

### Scale stats after 24

In [31]:
combined = pd.read_csv('..\\..\\Data\\stata-files\\combined.csv')
combined.dropna(subset=['age'], inplace=True)

In [32]:
## Scale by position
scaler=StandardScaler()
pos_scaled=pd.DataFrame()
for pos in combined['position'].unique():
    temp=combined[combined['position']==pos].copy()
    temp['seasons_over_24']=scaler.fit_transform(temp[['seasons_over_24']])
    temp['age_at_retirement']=scaler.fit_transform(temp[['age_at_retirement']])
    temp['games_over_24']=scaler.fit_transform(temp[['games_over_24']])
    pos_scaled=pd.concat([pos_scaled, temp])

In [33]:
## Scale by round
scaler=StandardScaler()
round_scaled=pd.DataFrame()
for round in combined['round'].unique():
    temp=combined[combined['round']==round].copy()
    temp['seasons_over_24']=scaler.fit_transform(temp[['seasons_over_24']])
    temp['age_at_retirement']=scaler.fit_transform(temp[['age_at_retirement']])
    temp['games_over_24']=scaler.fit_transform(temp[['games_over_24']])
    round_scaled=pd.concat([round_scaled, temp])

In [34]:
## Scale by round and position
scaler=StandardScaler()
scaled=pd.DataFrame()
for round in pos_scaled['round'].unique():
    temp=pos_scaled[pos_scaled['round']==round].copy()
    temp['seasons_over_24']=scaler.fit_transform(temp[['seasons_over_24']])
    temp['age_at_retirement']=scaler.fit_transform(temp[['age_at_retirement']])
    temp['games_over_24']=scaler.fit_transform(temp[['games_over_24']])
    scaled=pd.concat([scaled, temp])

In [35]:
combined['pos_seasons_over_24']=pos_scaled['seasons_over_24']
combined['pos_age_at_retirement']=pos_scaled['age_at_retirement']
combined['pos_games_over_24']=pos_scaled['games_over_24']

combined['round_seasons_over_24']=round_scaled['seasons_over_24']
combined['round_age_at_retirement']=round_scaled['age_at_retirement']
combined['round_games_over_24']=round_scaled['games_over_24']

combined['scaled_seasons_over_24']=scaled['seasons_over_24']
combined['scaled_age_at_retirement']=scaled['age_at_retirement']
combined['scaled_games_over_24']=scaled['games_over_24']

In [36]:
combined.sort_values(by=['season', 'name'], inplace=True)
combined.to_csv('..\\..\\Data\\stata-files\\combined.csv',index=False)

## Added Season Data

In [15]:
player_stats = pd.read_csv('..\\..\\Data\\NHL_Players_Statistics.csv')
nhl_draft = pd.read_csv('..\\..\\Data\\stata-files\\nhl_draft.csv')
combined = pd.read_csv('..\\..\\Data\\stata-files\\combined.csv')

combined['team']=combined['team'].str.replace('.', '')

combined_teams = combined['team'].unique()
player_stats_teams = player_stats['TEAM'].unique()



In [41]:
a=player_stats['SEASON_year'].unique().tolist()
a.sort()
a

[1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]

In [26]:
player_stats_teams

array(['BOS', 'CHI', 'DAL', 'MTL', 'STL', 'DET', 'WPG', 'TB', 'CGY',
       'MIN', 'TOR', 'ANA', 'COL', 'QUE', 'LA', 'VAN', 'NYI', 'WSH',
       'ARI', 'CAR', 'CBJ', 'NYR', 'EDM', 'ATL', 'SJ', 'HAR', 'PHI',
       'BUF', 'NSH', 'PIT', 'NJ', 'PHX', 'FLA', 'VGS', 'OTT'],
      dtype=object)

In [33]:
teams=nhl_draft['team'].unique()

In [34]:
for team in teams:
    if team not in player_stats_teams:
        print(team)

SEA
nan


In [36]:
for team in player_stats_teams:
    if team not in teams:
        print(team)

In [3]:
nhl_draft['team'].replace({'Atlanta Thrashers':'ATL', 'Montreal Canadiens':'MTL', 'Edmonton Oilers':'EDM', 'Toronto Maple Leafs':'TOR',
                           'New Jersey Devils':'NJ', 'Arizona Coyotes': 'ARI', 'Seattle Kraken': 'SEA','Philadelphia Flyers':'PHI',
                           'Columbus Blue Jackets':'CBJ', 'New York Islanders':'NYI', 'New York Rangers':'NYR', 'Florida Panthers':'FLA',
                           'Chicago Blackhawks':'CHI','Detroit Red Wings':'DET','Buffalo Sabres': 'BUF', 'Anaheim Ducks':'ANA',
                           'Winnipeg Jets':'WPG', 'Vancouver Canucks':'VAN', 'Nashville Predators':'NSH', 'Dallas Stars':'DAL', 'Minnesota Wild':'MIN',
                           'Washington Capitals':'WSH', 'Pittsburgh Penguins':'PIT', 'St. Louis Blues':'STL', 'San Jose Sharks':'SJ', 'Tampa Bay Lightning':'TB',
                           'Vegas Golden Knights':'VGS','Los Angeles Kings':'LA', 'Boston Bruins':'BOS','Calgary Flames':'CGY', 'Carolina Hurricanes': 'CAR',
                           'Ottawa Senators':'OTT', 'Colorado Avalanche':'COL', 'Quebec Nordiques':'QUE', 'Hartford Whalers':'HAR', 'Phoenix Coyotes': 'PHX',
                           'Colorado Rockies':'COL', 'Minnesota North Stars':'MIN' }, inplace=True)

### Preprocess names to create player ID

In [ ]:
def remove_punctuation(text):

    assert type(text)==str, 'Input must be a string:' + str(text)
    return "".join(i for i in text if i not in string.punctuation)

player_stats.dropna(subset=['Name'], inplace=True)
nhl_draft.dropna(subset=['player','team'], inplace=True)

for index, row in player_stats.iterrows():
    name=remove_punctuation(row['Name'])
    name=name.upper()
    player_stats.at[index,'Name']=name
    player_stats.at[index,'ID']=name+row['TEAM']

for index, row in nhl_draft.iterrows():
    name=remove_punctuation(row['player'])
    name=name.upper()
    nhl_draft.at[index,'player']=name
    nhl_draft.at[index,'ID']=name+row['team']






In [13]:
no_overlap=[]

for index, row in nhl_draft.iterrows():
    if row['ID'] not in player_stats['ID'].values:
        no_overlap.append(row['ID'])

len(no_overlap)

8105

In [14]:
no_overlap=[]

for index, row in player_stats.iterrows():
    if row['ID'] not in nhl_draft['ID'].values:
        no_overlap.append(row['ID'])

len(no_overlap)


18025

In [16]:
names=[]
for name in player_stats['Name'].unique():
     if name not in nhl_draft['player'].unique():
        names.append(name)

len(names)

611

In [22]:
for name in player_stats['Name'].unique():
    temp = player_stats[player_stats['Name']==name]
    if temp['SEASON_year'].value_counts().max()>1:
        temp = temp[temp['SEASON_year'].value_counts()>1]
        for year in temp['SEASON_year'].unique():
            if temp['Experience'].value_counts().max()>1:
                print(name, year)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [19]:
player_stats[player_stats['Name']=='Aaron Downey']

,Name,Date_of_birth,SEASON_year,SEASON,TEAM,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,...,Saves,Save_Percentage,Shutouts,Position,Height,Weight,Body_mass_index,Place_of_birth,Age,Experience
0,Aaron Downey,1974-08-27,2000,'99-'00,BOS,1,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,26,1
1,Aaron Downey,1974-08-27,2001,'00-'01,CHI,3,0.0,0.0,0.0,-1.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,27,2
2,Aaron Downey,1974-08-27,2002,'01-'02,CHI,36,1.0,0.0,1.0,-2.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,28,3
3,Aaron Downey,1974-08-27,2003,'02-'03,DAL,43,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,29,4
4,Aaron Downey,1974-08-27,2004,'03-'04,DAL,37,1.0,1.0,2.0,2.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,30,5
5,Aaron Downey,1974-08-27,2006,'05-'06,MTL,25,1.0,4.0,5.0,2.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,32,7
6,Aaron Downey,1974-08-27,2006,'05-'06,STL,17,2.0,0.0,2.0,0.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,32,7
7,Aaron Downey,1974-08-27,2007,'06-'07,MTL,21,1.0,0.0,1.0,-6.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,33,8
8,Aaron Downey,1974-08-27,2008,'07-'08,DET,56,0.0,3.0,3.0,0.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,34,9
9,Aaron Downey,1974-08-27,2009,'08-'09,DET,4,1.0,1.0,2.0,0.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,Shelburne. Ontario,35,10


# NFL

In [2]:
nfl_draft=pd.read_csv('..\\..\\Data\\stata-files\\nfl_draft.csv')
nfl_draft.head(32)

,season,round,overall_pick,team,gsis_id,pfr_player_id,cfb_player_id,pfr_player_name,hof,position,...,position_code,pos_years,pos_age_of_retirement,pos_games_played,round_years,round_age_of_retirement,round_games_played,scaled_years,scaled_age_of_retirement,scaled_games_played
0,1980,1,1,DET,NaN,SimsBi00,billy-sims-1,Billy Sims,False,RB,...,19.0,-0.031224,0.497970,0.078532,-0.860013,-0.346221,-0.805581,-0.684205,-0.058423,-0.659695
1,1980,1,2,NYJ,NaN,JoneLa00,lam-jones-1,Lam Jones,False,WR,...,23.0,-0.078148,-0.179959,0.032186,-0.860013,-0.872640,-0.787929,-0.730463,-0.707512,-0.705119
2,1980,1,3,CIN,NaN,MunoAn00,NaN,Anthony Munoz,True,T,...,21.0,1.766940,1.609314,2.001237,1.298320,1.233034,1.400951,1.088438,1.005643,1.224742
3,1980,1,4,GNB,NaN,ClarBr23,bruce-clark-1,Bruce Clark,False,DE,...,3.0,1.071033,0.940618,0.728635,0.488945,0.443406,0.129988,0.402408,0.365394,-0.022532
4,1980,1,5,BAL,NaN,DickCu00,curtis-dickey-1,Curtis Dickey,False,RB,...,19.0,0.611853,0.823669,0.625120,-0.320429,-0.083012,-0.364275,-0.050254,0.253421,-0.123986
5,1980,1,6,STL,NaN,GreeCu21,curtis-greer-1,Curtis Greer,False,DE,...,3.0,0.526144,0.388654,0.398066,-0.050638,-0.083012,-0.205404,-0.134748,-0.163089,-0.346521
6,1980,1,7,ATL,NaN,MillJu00,junior-miller-1,Junior Miller,False,TE,...,22.0,-0.239124,-0.391103,0.047253,-0.860013,-0.872640,-0.611406,-0.889155,-0.909673,-0.690352
7,1980,1,8,NYG,NaN,HaynMa00,mark-haynes-1,Mark Haynes,False,DB,...,2.0,1.166142,0.765859,1.048311,0.488945,0.180197,0.377120,0.496167,0.198070,0.290782
8,1980,1,9,MIN,NaN,MartDo21,NaN,Doug Martin,False,DE,...,3.0,1.071033,1.216600,0.954813,0.488945,0.706615,0.359468,0.402408,0.629635,0.199145
9,1980,1,10,SEA,NaN,GreeJa01,jacob-green-2,Jacob Green,False,DE,...,3.0,1.888368,2.044546,1.894322,1.298320,1.496243,1.312690,1.208143,1.422360,1.119955


In [3]:
nfl_draft.columns

Index(['season', 'round', 'overall_pick', 'team', 'gsis_id', 'pfr_player_id',
       'cfb_player_id', 'pfr_player_name', 'hof', 'position', 'category',
       'side', 'college', 'age', 'to', 'allpro', 'probowls', 'seasons_started',
       'w_av', 'car_av', 'dr_av', 'games', 'pass_completions', 'pass_attempts',
       'pass_yards', 'pass_tds', 'pass_ints', 'rush_atts', 'rush_yards',
       'rush_tds', 'receptions', 'rec_yards', 'rec_tds', 'def_solo_tackles',
       'def_ints', 'def_sacks', 'pick', 'distRound', 'seasonsPlayed',
       'age_of_retirement', 'position_code', 'pos_years',
       'pos_age_of_retirement', 'pos_games_played', 'round_years',
       'round_age_of_retirement', 'round_games_played', 'scaled_years',
       'scaled_age_of_retirement', 'scaled_games_played'],
      dtype='object')

## Create pick from overall pick

In [ ]:
teams=nfl_draft['round'].to_numpy()
prev=0
pick=1
picks=[]
for p in teams:
    if p==prev:
        pick+=1
    else:
        pick=1

    prev=p
    picks.append(pick)


In [ ]:
nfl_draft=nfl_draft.rename(columns={'pick':'overall_pick'})
nfl_draft['pick']=picks

## Recenter data and create running variable distRound

In [ ]:
distRound=[]
for index, row in nfl_draft.iterrows():
    numPicks=nfl_draft[(nfl_draft['season']==row['season']) & (nfl_draft['round']==row['round'])]['pick'].max()
    m=numPicks//2
    if row['pick']>m:
        distRound.append(row['pick']-numPicks)
    else:
        distRound.append(row['pick'])

nfl_draft['distRound']=distRound


## Create variables and encode positions

In [ ]:
nfl_draft['seasonsPlayed']=nfl_draft['to']-nfl_draft['season']
nfl_draft['age_of_retirement']=nfl_draft['age']+nfl_draft['seasonsPlayed']
ord_enc = OrdinalEncoder()
nfl_draft["position_code"] = ord_enc.fit_transform(nfl_draft[["position"]])


## Standardize the data

In [4]:
## Scale by position
scaler=StandardScaler()
pos_scaled_nfl_draft=pd.DataFrame()
for pos in nfl_draft['position'].unique():
    temp=nfl_draft[nfl_draft['position']==pos].copy()
    temp['seasonsPlayed']=scaler.fit_transform(temp[['seasonsPlayed']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games']=scaler.fit_transform(temp[['games']])
    temp['seasons_started']= scaler.fit_transform(temp[['seasons_started']])
    pos_scaled_nfl_draft=pd.concat([pos_scaled_nfl_draft, temp])

In [5]:
## Scale by round
scaler=StandardScaler()
round_scaled_nfl_draft=pd.DataFrame()
for round in nfl_draft['round'].unique():
    temp=nfl_draft[nfl_draft['round']==round].copy()
    temp['seasonsPlayed']=scaler.fit_transform(temp[['seasonsPlayed']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games']=scaler.fit_transform(temp[['games']])
    temp['seasons_started']=scaler.fit_transform(temp[['seasons_started']])
    round_scaled_nfl_draft=pd.concat([round_scaled_nfl_draft, temp])

In [6]:
## Scale by round and position
scaler=StandardScaler()
scaled_nfl_draft=pd.DataFrame()
for round in pos_scaled_nfl_draft['round'].unique():
    temp=pos_scaled_nfl_draft[pos_scaled_nfl_draft['round']==round].copy()
    temp['seasonsPlayed']=scaler.fit_transform(temp[['seasonsPlayed']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games']=scaler.fit_transform(temp[['games']])
    temp['seasons_started']=scaler.fit_transform(temp[['seasons_started']])
    scaled_nfl_draft=pd.concat([scaled_nfl_draft, temp])

In [7]:
nfl_draft['pos_years']=pos_scaled_nfl_draft['seasonsPlayed']
nfl_draft['pos_age_of_retirement']=pos_scaled_nfl_draft['age_of_retirement']
nfl_draft['pos_games_played']=pos_scaled_nfl_draft['games']
nfl_draft['pos_seasons_started']=pos_scaled_nfl_draft['seasons_started']


nfl_draft['round_years']=round_scaled_nfl_draft['seasonsPlayed']
nfl_draft['round_age_of_retirement']=round_scaled_nfl_draft['age_of_retirement']
nfl_draft['round_games_played']=round_scaled_nfl_draft['games']
nfl_draft['round_seasons_started']=round_scaled_nfl_draft['seasons_started']


nfl_draft['scaled_years']=scaled_nfl_draft['seasonsPlayed']
nfl_draft['scaled_age_of_retirement']=scaled_nfl_draft['age_of_retirement']
nfl_draft['scaled_games_played']=scaled_nfl_draft['games']
nfl_draft['scaled_seasons_started']=scaled_nfl_draft['seasons_started']


## Write to CSV

In [8]:
nfl_draft.to_csv('..\\..\\Data\\stata-files\\nfl_draft.csv',index=False)

In [ ]:
nhl_draft=pd.read_csv('..\\..\\Data\\stata-files\\nhl_draft.csv')
nhl_draft